### Extending bidirectional attention for LLMs via ULLME. 

In [ ]:
from ullme.models import ULLME

model = ULLME(
            model_name_or_path="microsoft/phi-1_5",
            model_backbone_type="phi",
            )
model.cuda()
print("Model Architecture: ")
print(model)

We also support LoRA patching for parameter-effecient fine-tuning 

In [ ]:
from ullme.models import ULLME

lora_model = ULLME(
            model_name_or_path="microsoft/phi-1_5",
            model_backbone_type="phi",
            lora_name="ullme-phi",
            loar_r=16,
            lora_alpha=32,
            )
lora_model.cuda()
print("Model Architecture: ")
print(lora_model)

Compute sequence representaion with Bidirectional Extended LLMs

In [ ]:
input_sentence = "This a example sentence."
model_inputs = model.tokenizer(
                            [input_sentence],
                            return_tensors='pt'
                            )
model_output = model(
                    input_ids=model_inputs['input_ids'].cuda(),
                    attention_mask=model_inputs['attention_mask'].cuda(),
                    is_generate=False
                    )
reps = model_output['reps']
print("Reps Shape: ", reps.shape)
print("Reps: ", reps)

### Evaluation MTEB dataset via ULLME.

Here, we support almost LLM models available in HF. For example, we try to use top1 model in MTEB (dunzhang/stella_en_1.5B_v5)

In [ ]:
from ullme.models import WrappedULLME
from ullme.eval import eval_mteb_dataset


model = WrappedULLME(model_name_or_path='dunzhang/stella_en_1.5B_v5')
print("Model Architecture: ")
print(model)

After loading the model, you need to select specific datasets and language subsets for evaluation. 

In [ ]:
eval_result = eval_mteb_dataset(
                                model=model,
                                dataset_name='ArguAna',
                                langs=['eng'],
                                )
print("Eval Result: ", eval_result)

### Fine-tune LLMs with ULLME

We support various training strategies including Constrastive Loss, SFT, DPO and GRL. The following spinet inlustrate how to use ULLME for fine-tuning LLM for Dense Retrieval. 

``` python
from ullme.trainer import GradCacheTrainer
trainer = GradCacheTrainer(
                            con_loss_type='NTXentLoss',
                            gen_loss_type='sigmoid', # 'sft'
                            use_kl_loss=True
                            )
trainer.fit_epoch(
                model=model,
                train_loader=train_dataloader,
                )
```

Besides, ULLME also support GradCache, Cross-devices Constrastive loss, Multi-GPUs training, and orther rich features for further improve the training process. Please refer to the documentation and file ```ullme/train.py``` for further information. 